<a href="https://colab.research.google.com/github/subhadeepCh/MCQ-Generator/blob/main/MCQ_GENERATION_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCQ GENERATOR MODEL VERSION 2.0
* Created by Subhadeep Choudhuri

In [ ]:
# Downloading tools
!pip install wikipedia
!pip install spacy
!pip install pytextrank
!pip install rake-nltk

In [153]:
# Importing all tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from nltk.corpus import brown
from nltk.data import find
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import pytextrank
import rake_nltk
import spacy
import gensim
import wikipedia
from rake_nltk import Rake
import re
from nltk.corpus import wordnet

In [154]:
#downloads
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('word2vec_sample')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Input Text Document

In [8]:
data=''
print('Do You Want to import a wikipedia article or input your path')
print('input 1 for first option and 2 for second option')
choice=input('Enter Choice')
if choice=='1':
  title=input('wikipedia title')
  data = wikipedia.page(f'title={title}').content
  print('Length of data = '+str(len(data)))
  print("\n-------\n"+data[:50]+"\n-------\n")
else:
  path=input('path')
  with open(path, "r") as file1:
    data = file1.read()
  print('Length of data = '+str(len(data)))
  print("\n-------\n"+data[:50]+"\n-------\n")

Do You Want to import a wikipedia article or input your path
input 1 for first option and 2 for second option
Enter Choice1
wikipedia titleHistory of India
Length of data = 116737

-------
According to consensus in modern genetics, anatomi
-------



# Summarizing the passage

In [26]:
def summarize_text(input_text='Example data',phrase_num='Number of phrases',num='Enter number of questions'):
  """
  Enter Data to get its summary.
  Each sentence of the summary will be 1 question
  """
  if (type(num)!=type(1) and type(phrase_num)!=type(1)):
    phrase_num=1
    num=1
  print(f"Number of phrases = {phrase_num} \n")
  print(f"Number of questions = {num} \n")
  # Configuring the pipeline
  nlp = spacy.load("en_core_web_sm")
  tr = pytextrank.TextRank()
  nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
  # Adding the data
  doc = nlp(input_text)
  # summarize the document based on the top 15 phrases, 
  # yielding the top 5 sentences...
  sentences=[]
  for sent in doc._.textrank.summary(limit_phrases=int(phrase_num), limit_sentences=int(num)):
    sentences.append(str(sent))
  c=1
  for q in sentences:
    print(str(c)+". "+q+"\n")
    c+=1
  return sentences

In [27]:
sentences=summarize_text(input_text=data,phrase_num=100,num=30)

Number of phrases = 100 

Number of questions = 30 

1. The first of these was the outright annexation of Indian states and subsequent direct governance of the underlying regions, which collectively came to comprise British India.  

2. The south Indian mathematician Madhava of Sangamagrama founded the famous Kerala School of Astronomy and Mathematics in the 14th century which produced a lot of great south Indian mathematicians like Parameshvara, Nilakantha Somayaji and Jyeṣṭhadeva in medieval south India.

3. Chandragupta rapidly expanded his power westwards across central and western India, and by 317 BCE the empire had fully occupied Northwestern India.  

4. Although these continental European powers controlled various coastal regions of southern and eastern India during the ensuing century, they eventually lost all their territories in India to the British, with the exception of the French outposts of Pondichéry and Chandernagore, and the Portuguese colonies of Goa, Daman and Diu.

# Function to find keywords from each sentence

In [104]:
def find_keywords(string='Enter question'):
  """
  Function to return list for most important keywords
  """
  r=Rake()
  # string = re.sub(r'[^\w\s]', '', string)
  r.extract_keywords_from_text(string)
  return r.get_ranked_phrases_with_scores()

In [105]:
def keyword(string='Enter question'):
  """
  Incresing keyword efficiency
  """
  key=find_keywords(string=string)
  count=[]
  for i in range(len(key)):
    count.append(int(0))
  pos=[]
  for item in key:
    t=word_tokenize(item[1])
    pos.append(nltk.pos_tag(t))
    print(nltk.pos_tag(t))
    # print("\n")
  i=0
  for i in range(len(pos)):
    t=0
    for items in pos[i]:
      if items[1] == 'CD':
        count[i]+=10
      elif items[1] == 'NNP':
        count[i]+=7
      elif items[1] == 'NNS':
        count[i]+=5
      elif items[1] == 'NN':
        count[i]+=4
      else :
        pass
    i+=1
  print(count)
  print(f"Largest Index at {np.argmax(count)}")
  most_imp=''
  phrase=pos[np.argmax(count)]
  found_count=0
  # find adjective and noun pair
  itr=-1
  for items in phrase:
    if items[1]=='CD':
      found_count=1
      most_imp=str(items[0])
      break
  if found_count ==0:
    for items in phrase:
       itr+=1
       if items[1]=='JJ':
          if (itr+1<len(phrase)):
             if phrase[itr+1][1][0]=='N':
                found_count=1
                most_imp=str(items[0])+" "+str(phrase[itr+1][0]).capitalize()
                break
   
  if found_count == 0:
    for items in phrase:
      if items[1][0]=='N':
        found_count=1
        most_imp=str(items[0]).capitalize()
        break
  if found_count == 0:
    most_imp=phrase[0][0]
  return most_imp  


# Generating fill in the blanks

In [138]:
def create_mcq(sen):
  answer_list=[]
  for questions in sen:
    answer_list.append(keyword(questions))
  questions=sen
  bank={}
  i=0
  for i in range(len(questions)):
    lower=questions[i].lower().find(answer_list[i].lower())
    upper=lower+len(answer_list[i])
    quest=questions[i][:lower] + str('_________') + questions[i][upper:]
    ans=answer_list[i]
    bank[quest]=ans;
    banknew={'Question':bank.keys(),'Answers':bank.values()}
  return banknew
    

In [141]:
df=pd.DataFrame.from_dict(create_mcq(sentences))
df

[('subsequent', 'JJ'), ('direct', 'JJ'), ('governance', 'NN')]
[('comprise', 'NN'), ('british', 'JJ'), ('india', 'NN')]
[('underlying', 'JJ'), ('regions', 'NNS')]
[('outright', 'JJ'), ('annexation', 'NN')]
[('indian', 'JJ'), ('states', 'NNS')]
[('collectively', 'RB'), ('came', 'VBD')]
[('first', 'RB')]
[4, 8, 5, 4, 5, 0, 0]
Largest Index at 1
[('great', 'JJ'), ('south', 'JJ'), ('indian', 'JJ'), ('mathematicians', 'NNS'), ('like', 'IN'), ('parameshvara', 'NN')]
[('south', 'JJ'), ('indian', 'JJ'), ('mathematician', 'NN'), ('madhava', 'NN')]
[('medieval', 'NN'), ('south', 'NN'), ('india', 'NN')]
[('famous', 'JJ'), ('kerala', 'CD'), ('school', 'NN')]
[('sangamagrama', 'NN'), ('founded', 'VBD')]
[('nilakantha', 'NN'), ('somayaji', 'NN')]
[('14th', 'JJ'), ('century', 'NN')]
[('produced', 'VBN')]
[('mathematics', 'NNS')]
[('lot', 'NN')]
[('jyeṣṭhadeva', 'NN')]
[('astronomy', 'NN')]
[9, 8, 12, 14, 4, 8, 4, 0, 5, 4, 4, 4]
Largest Index at 3
[('power', 'NN'), ('westwards', 'NNS'), ('across', 'IN

,Question,Answers
0,The first of these was the outright annexation...,british India
1,The south Indian mathematician Madhava of Sang...,kerala
2,Chandragupta rapidly expanded his power westwa...,317
3,Although these continental _________ controlle...,european Powers
4,"This period also covers the ""Late Classical Ag...",Rajendra
5,An Economic History of India: From Pre-Colonia...,Oxford
6,The region of Bengal in British India suffered...,1940
7,Muslim League leader Muhammad Ali Jinnah procl...,16
8,"Under the Cholas, the South India reached ____...",new Heights
9,The subsequent Slave dynasty of Delhi managed ...,principal Hindu


In [142]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Minor Project V1/generated2.csv')

In [155]:
ans_list=np.array(df["Answers"])

# Using Word2Vec

In [145]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [146]:
len(model.vocab)

43981

In [147]:
# Each word is represented in the space of 300 dimensions:
len(model['university'])

300

In [152]:
model.most_similar(positive=['university'], topn = 3)

[('universities', 0.7003918886184692),
 ('faculty', 0.6780906915664673),
 ('undergraduate', 0.6587096452713013)]

In [187]:
distractors=[]
for anss in ans_list:
  synonyms=[]
  abc=str(word_tokenize(anss)[0])
  try :
    item=model.most_similar(positive=[abc], topn = 100)
    ps = PorterStemmer() 
    for i in item:
      if str(ps.stem(i)).lower() != str(ps.stem(abc)).lower():
        synonyms.append(i[0])
  except:    
    for syn in wordnet.synsets(abc):
      for l in syn.lemmas():
        if (l.name().lower() != abc):
          if(ps.stem(l.name())!=ps.stem(ps.stem(abc))):
            synonyms.append(l.name())
        else :
          if l.antonyms() and l.antonyms()[0].name().lower() !=abc:
            synonyms.append(l.antonyms()[0].name())
          else :
            pass
    
  dist=[]
  for d in synonyms :
    tmp=anss
    tmp=anss.replace(abc,d)
    dist.append(tmp)
  try:
    a=int(anss)
    dist.append('1934')
    dist.append('1789')
    dist.append('2000')
  except:
    pass
  dist.append('None of the above')
  distractors.append(dist)
print(distractors)


[['British_people India', 'Brits India', 'None of the above'], ['None of the above'], ['1934', '1789', '2000', 'None of the above'], ['None of the above'], ['None of the above'], ['Oxford_University', 'None of the above'], ['1934', '1789', '2000', 'None of the above'], ['sixteen', 'XVI', 'sixteen', 'xvi', '1934', '1789', '2000', 'None of the above'], ['old Heights', 'fresh Heights', 'novel Heights', 'raw Heights', 'unexampled Heights', 'worn Heights', 'newfangled Heights', 'Modern Heights', 'young Heights', 'newly Heights', 'freshly Heights', 'fresh Heights', 'None of the above'], ['school_principal Hindu', 'head_teacher Hindu', 'head Hindu', 'star Hindu', 'lead Hindu', 'corpus Hindu', 'principal_sum Hindu', 'dealer Hindu', 'chief Hindu', 'main Hindu', 'primary Hindu', 'master Hindu', 'None of the above'], ['Turki', 'Turko-Tatar', 'Turkic_language', 'None of the above'], ['Republic_of_India Company', 'Bharat Company', 'None of the above'], ['fifteenth Century', 'None of the above'], ['

In [188]:
df=df.assign(distractors=distractors)
df

,Question,Answers,distractors
0,The first of these was the outright annexation...,british India,"[British_people India, Brits India, None of th..."
1,The south Indian mathematician Madhava of Sang...,kerala,[None of the above]
2,Chandragupta rapidly expanded his power westwa...,317,"[1934, 1789, 2000, None of the above]"
3,Although these continental _________ controlle...,european Powers,[None of the above]
4,"This period also covers the ""Late Classical Ag...",Rajendra,[None of the above]
5,An Economic History of India: From Pre-Colonia...,Oxford,"[Oxford_University, None of the above]"
6,The region of Bengal in British India suffered...,1940,"[1934, 1789, 2000, None of the above]"
7,Muslim League leader Muhammad Ali Jinnah procl...,16,"[sixteen, XVI, sixteen, xvi, 1934, 1789, 2000,..."
8,"Under the Cholas, the South India reached ____...",new Heights,"[old Heights, fresh Heights, novel Heights, ra..."
9,The subsequent Slave dynasty of Delhi managed ...,principal Hindu,"[school_principal Hindu, head_teacher Hindu, h..."


In [189]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Minor Project V1/generated_with_distractors2.csv')

In [190]:
import random
for i in range(len(df)):
  print(str(i+1)+'. '+df['Question'][i])
  mcq=[]
  mcq.append(df['Answers'][i])
  for item in df['distractors'][i][:3]:
    mcq.append(item)
  random.shuffle(mcq)
  if(len(mcq)>=1):
    print('\na) '+mcq[0]+"\n")
  if(len(mcq)>=2):
    print('b) '+mcq[1]+"\n")
  if(len(mcq)>=3):
    print('c) '+mcq[2]+"\n")
  if(len(mcq)>=4):
     print('d) '+mcq[3]+"\n")
  print('\n')

1. The first of these was the outright annexation of Indian states and subsequent direct governance of the underlying regions, which collectively came to comprise _________.  

a) british India

b) None of the above

c) Brits India

d) British_people India



2. The south Indian mathematician Madhava of Sangamagrama founded the famous _________ School of Astronomy and Mathematics in the 14th century which produced a lot of great south Indian mathematicians like Parameshvara, Nilakantha Somayaji and Jyeṣṭhadeva in medieval south India.

a) None of the above

b) kerala



3. Chandragupta rapidly expanded his power westwards across central and western India, and by _________ BCE the empire had fully occupied Northwestern India.  

a) 317

b) 1934

c) 2000

d) 1789



4. Although these continental _________ controlled various coastal regions of southern and eastern India during the ensuing century, they eventually lost all their territories in India to the British, with the exception of th